In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import time
from collections import defaultdict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import re

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

import random
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random


In [2]:
browser = webdriver.Chrome()

## 🔐 Automating Amazon Login Using Selenium

This script automates the login process on Amazon.in using Selenium WebDriver. It opens the login page, enters the phone number and password with human-like delays, and clicks the necessary buttons (Continue and Sign-In) to complete the login process.


In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

phone_no = "1234567890"
password = "xyz"

login_link = "https://www.amazon.in/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0"

browser.get(login_link)

wait = WebDriverWait(browser, 15)
actions = ActionChains(browser)

def slow_type(element, text, delay_range=(0.05, 0.15)):
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(*delay_range))

wait.until(EC.presence_of_element_located((By.ID, 'authportal-main-section')))
time.sleep(random.uniform(1.5, 3))

try:
    try:
        email_input = wait.until(EC.visibility_of_element_located((By.ID, 'ap_email')))
    except:
        email_input = wait.until(EC.visibility_of_element_located((By.ID, 'ap_email_login')))
        
    email_input.click()
    time.sleep(random.uniform(0.2, 0.5))
    email_input.clear()
    slow_type(email_input, phone_no)
    print("✅ Phone/email entered.")
except Exception as e:
    print("❌ Could not enter phone/email:", e)

time.sleep(random.uniform(1.5, 3))

try:
    continue_btn = wait.until(EC.element_to_be_clickable((By.ID, 'continue')))
    continue_btn.click()
    print("✅ Clicked Continue.")
except Exception as e:
    print("❌ Failed to click Continue:", e)

time.sleep(random.uniform(1.5, 3))

try:
    password_input = wait.until(EC.visibility_of_element_located((By.ID, 'ap_password')))
    password_input.click()
    time.sleep(random.uniform(0.2, 0.5))
    password_input.clear()
    slow_type(password_input, password)
    print("✅ Password entered.")
except Exception as e:
    print("❌ Could not enter password:", e)

time.sleep(random.uniform(1.5, 3))

try:
    sign_in_button = wait.until(EC.element_to_be_clickable((By.ID, 'signInSubmit')))
    actions.move_to_element(sign_in_button).pause(0.3).click().perform()
    print("✅ Clicked Sign-In.")
except Exception as e:
    print("❌ Could not click Sign-In:", e)


✅ Phone/email entered.
✅ Clicked Continue.
✅ Password entered.
✅ Clicked Sign-In.


## 📦 Scraping Product and Review Data from Amazon.in

This script automates the process of:

1. Taking a product name input (e.g., "mobile", "laptop") from the user.
2. Opening the Amazon search results page and listing available brands.
3. Letting the user select one or more brands to filter.
4. Navigating through all pages of product listings for the selected brand(s).
5. Visiting each product page to extract product details like title, price, rating, ASIN, model number, etc.
6. Opening the review section and scraping all reviews with metadata (rating, title, text, date, etc.).
7. Saving all collected data into a list for further processing.

It uses **Selenium** for browser automation and **BeautifulSoup** for parsing HTML content.


In [ ]:
product_detail_list = []
visited_review_links = set()


last_page = 1



try:
    while True:
        search_item = input("Enter product you want to search (like Mobile, laptop ....) - ").strip()
        if search_item.replace(" ", "").isalpha():
            break
        else:
            print("❌ Enter a valid search item (letters and spaces only).")
            time.sleep(random.uniform(0.8, 1.4))  # simulating rethinking time
    
    time.sleep(random.uniform(1.5, 2.5))
    first_url = 'https://www.amazon.in/s?k=' + search_item
    browser.get(first_url)

    scroll_pause_time = random.uniform(1.8, 2.5)
    max_scrolls = 10
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    for _ in range(max_scrolls):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(browser.page_source, 'html.parser')

    soup.find()
    
    try:
        filter_phone_brands = soup.find('div', id="brandsRefinements") \
                                   .find('ul', id="filter-p_123") \
                                   .find_all('span', attrs={"role": "presentation"})
        
        avl_brands = []
        avl_brands.append(["0", "all", first_url])
    
        i_x = 1
        for brand in filter_phone_brands[1:]:
            brand_name = brand.find('a').text.strip()
            updated_url = "https://www.amazon.in" + brand.find('a').get('href')
            avl_brands.append([str(i_x), brand_name, updated_url])
            i_x += 1
    
        brand_df = pd.DataFrame(data=avl_brands, columns=['Index', "Brand Name", "Url"])
    
        print("\n✅ You can search by Index or Brand Name\n")
        print(brand_df[['Index', "Brand Name"]].to_string(index=False))


        brand_name_list = [name.lower() for name in brand_df['Brand Name'].to_list()]

    
        while True:
            user_input = input("\nEnter Index or Brand Name you want to search - ").strip()

            user_input_split = list(set(user_input.split(',')))
            

            if len(user_input_split) > 1:
                
                if (all(val.strip() in brand_df['Index'].to_list() for val in user_input_split) or all(val.strip().lower() in brand_name_list for val in user_input_split)):


                    if user_input_split[0].strip().isdigit():
                        match = brand_df[brand_df['Index'] == user_input_split[0].strip()]
                        print("Brand Name - ",brand_df[brand_df['Index'] == user_input_split[0].strip()]['Brand Name'].values[0])
                        final_url = match['Url'].values[0]
                        
                    else:
                        match = brand_df[brand_df['Brand Name'].str.lower() == user_input_split[0].strip().lower()]
                        print("Brand Name - ",user_input_split[0].strip())
                        final_url = match['Url'].values[0]

                    
                    
                    for single_input in user_input_split[1:]:
                        browser.get(final_url)
    
                        scroll_pause_time = random.uniform(1.8, 2.5)
                        max_scrolls = 10
                        
                        last_height = browser.execute_script("return document.body.scrollHeight")
                        
                        for _ in range(max_scrolls):
                            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            time.sleep(scroll_pause_time)
                            new_height = browser.execute_script("return document.body.scrollHeight")
                            if new_height == last_height:
                                break
                            last_height = new_height

                        
                        
                        ini_soup = BeautifulSoup(browser.page_source, 'html.parser')
                        if single_input.strip().isdigit():
                            search_brand_name = brand_df[brand_df['Index'] == single_input.strip()]['Brand Name'].values[0]

                            loop_brand_search = ini_soup.find('div', id="brandsRefinements") \
                                   .find('ul', id="filter-p_123") \
                                   .find_all('span', attrs={"role": "presentation"})
                            
                            for search_brand in loop_brand_search[1:]:
                                brand_name = search_brand.find('a').text.strip()
                                if brand_name.lower() == search_brand_name.strip().lower():
                                    print("Brand Name - ",brand_name)
                                    final_url = "https://www.amazon.in" + search_brand.find('a').get('href')
                                    break

                        else:
                            loop_brand_search = ini_soup.find('div', id="brandsRefinements") \
                                   .find('ul', id="filter-p_123") \
                                   .find_all('span', attrs={"role": "presentation"})
                            
                            for search_brand in loop_brand_search[1:]:
                                brand_name = search_brand.find('a').text.strip()
                                if brand_name.lower() == single_input.strip().lower():
                                    print("Brand Name - ",brand_name)
                                    final_url = "https://www.amazon.in" + search_brand.find('a').get('href')
                                    break
                    break
          
                else:
                    print("❌ Invalid input. Please enter a valid index or brand name, all matching.")

            elif len(user_input_split) == 1:
                if user_input.strip().isdigit():
                    match = brand_df[brand_df['Index'] == user_input.strip()]
                    print("Brand Name - ",brand_df[brand_df['Index'] == user_input.strip()]['Brand Name'].values[0])
                    final_url = match['Url'].values[0]
                    break
                else:
                    match = brand_df[brand_df['Brand Name'].str.lower() == user_input.strip().lower()]
                    print("Brand Name - ",user_input.strip())
                    final_url = match['Url'].values[0]
                    break

            else:
                print("❌ Invalid input. Please enter a valid index or brand name.")

    except Exception as e:
        print("⚠️ Couldn't extract brand filters. Using default search URL.")
        final_url = first_url
    
    browser.get(final_url)
    
    scroll_pause_time = random.uniform(1.8, 2.5)
    max_scrolls = 10
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    for _ in range(max_scrolls):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    try:
        last_page_tag = soup.find('div', class_="a-section a-text-center s-pagination-container") \
                        .find('span', class_="s-pagination-item s-pagination-disabled")
        if last_page_tag:
            last_page = last_page_tag.text.strip()
    
        else:
            last_page = soup.find('ul', class_="a-unordered-list a-horizontal s-unordered-list-accessibility").find_all('li', recursive = False)[-2].text.strip()
    
    except:
        prin("Goes to Last Page Except")
        last_page = 1
    
    print("last page", last_page)
    
    for page in range(1, int(last_page) + 1):
        post_final_url = final_url + "&page=" + str(page)
        print("Page Url", post_final_url)
        browser.get(post_final_url)
    
        scroll_pause_time = random.uniform(1.8, 2.5)
        max_scrolls = 10
        
        last_height = browser.execute_script("return document.body.scrollHeight")
        
        for _ in range(max_scrolls):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
    
        soup1 = BeautifulSoup(browser.page_source, 'html.parser')
        try:
            all_product = soup1.find('div', class_="s-main-slot s-result-list s-search-results sg-row") \
                               .find_all('div', role="listitem", recursive=False)
        except:
            prin("No product found.")
            continue
    
        for product in all_product:
            try:
                title_div = product.find('div', attrs={"data-cy": "title-recipe"})
                all_links = title_div.find_all('a')
                
                product_link = None
                for p_link in all_links:
                    href = p_link.get('href')
                    if href and href.startswith("/"):
                        product_link = 'https://www.amazon.in' + href
                        break
                    else:
                        continue
                        
                
                if not product_link:
                    print("❌ No valid product link found in this product.")
                    continue
            
    
    
                browser.get(product_link)
            
                scroll_pause_time = random.uniform(1.8, 2.5)
                max_scrolls = 10
                
                last_height = browser.execute_script("return document.body.scrollHeight")
                
                for _ in range(max_scrolls):
                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(scroll_pause_time)
                    new_height = browser.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
            
                soup2 = BeautifulSoup(browser.page_source, 'html.parser')

                prod_title = price = overall_rating = rating_count = bought_last_month = None
                asin = model_number = manufacturer_name = generic_name = customer_say = insight = None
            
                try:
                    prod_title = soup2.find('h1', id="title").text.strip()
                except:
                    pass
            
                try:
                    price = soup2.find('span', class_="a-price-whole").text.strip()
                except:
                    pass
            
                try:
                    overall_rating = soup2.find('span', attrs={"data-hook": "rating-out-of-text"}).text.split()[0].strip()
                except:
                    pass
            
                try:
                    rating_count = soup2.find('span', attrs={"data-hook": "total-review-count"}).text.split()[0].strip()
                except:
                    pass

                try:
                    bought_last_month = soup2.find('span', id="social-proofing-faceout-title-tk_bought").find("span", class_="a-text-bold").text.split()[0].strip()
                except:
                    pass
    
                try:
                    for all_detail in soup2.find_all('table', class_="a-keyvalue prodDetTable"):
                        for ind_detail in all_detail.find_all('tr'):
                            th = ind_detail.find('th')
                            td = ind_detail.find('td')
                            if th and td and "asin" in th.text.strip().lower():
                                asin = td.text.strip()
                                break
                except:
                    pass
                
                try:
                    for all_detail in soup2.find_all('table', class_="a-keyvalue prodDetTable"):
                        for ind_detail in all_detail.find_all('tr'):
                            th = ind_detail.find('th')
                            td = ind_detail.find('td')
                            if th and td and "model number" in th.text.strip().lower():
                                model_number = td.text.strip()
                                break
                except:
                    pass
                
                try:
                    for all_detail in soup2.find_all('table', class_="a-keyvalue prodDetTable"):
                        for ind_detail in all_detail.find_all('tr'):
                            th = ind_detail.find('th')
                            td = ind_detail.find('td')
                            if th and td and "brand name" in th.text.strip().lower():
                                manufacturer_name = td.text.strip()
                                break
                except:
                    pass
                
                try:
                    for all_detail in soup2.find_all('table', class_="a-keyvalue prodDetTable"):
                        for ind_detail in all_detail.find_all('tr'):
                            th = ind_detail.find('th')
                            td = ind_detail.find('td')
                            if th and td and "item type" in th.text.strip().lower():
                                generic_name = td.text.strip()
                                break
                except:
                    pass
                
                try:
                    customer_say = soup2.find('div', id="product-summary").find("p", class_="a-spacing-small").text.strip()
                
                except:
                    pass
                
                try:
                    insight_soup = soup2.find("div", attrs={"data-hook":"cr-insights-widget-aspects"}) \
                    .find('div', attrs={'aria-label': 'Commonly Mentioned Aspects'}) \
                    .find_all('a', recursive=False)
                    all_insight = []
                    for ind_insight in insight_soup:
                        all_insight.append(ind_insight.text.strip())
                
                    insight = '|'.join(all_insight)
                except:
                    pass

                print("ASIN, Model Number, manufacturer_name, generic_name", asin, model_number, manufacturer_name, generic_name)
            
                try:
                    review_link = soup2.find('a', attrs={"data-hook": "see-all-reviews-link-foot"}).get('href')
                    review_link = "https://www.amazon.in/" + review_link
                    
                except:
                    print("❌ No valid Review link found in this product.")
                    product_detail_list.append([asin, model_number, manufacturer_name, generic_name,
                    prod_title, price, overall_rating, rating_count, None, bought_last_month, product_link, customer_say, insight,
                    None, None, None,
                    None, False, None,   
                    None                 
                ])
    
                    continue
    
                if review_link not in visited_review_links:
                    visited_review_links.add(review_link)
                    browser.get(review_link)
                    time.sleep(random.uniform(2, 4))
                
                else:
                    print("Same Product")
                    continue
            
                while True:
                    scroll_pause_time = random.uniform(1.8, 2.5)
                    max_scrolls = 10
                    
                    last_height = browser.execute_script("return document.body.scrollHeight")
                    
                    for _ in range(max_scrolls):
                        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(scroll_pause_time)
                        new_height = browser.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height

                    soup4 = BeautifulSoup(browser.page_source, 'html.parser')

                    review_count = None
                    
                    try:
                        review_count = soup4.find('div', attrs={"data-hook":"cr-filter-info-review-rating-count"}).text.strip().split()[0].strip()
                    except:
                        pass
                        
                    all_review = soup4.find_all('li', attrs={"data-hook": "review"})
            
                    for ind_review in all_review:
                        try:
                            rating = ind_review.find('h5').find('i', attrs={"data-hook": "review-star-rating"}).text.split()[0].strip()
                        except:
                            rating = None
            
                        try:
                            review_title = ind_review.find('h5').find('span', class_=None).text.strip()
                        except:
                            review_title = None
            
                        try:
                            review_date = ind_review.find('span', attrs={"data-hook": "review-date"}).text.strip()
                        except:
                            review_date = None
            
                        try:
                            format_tag = ind_review.find('a', attrs={"data-hook": "format-strip"})
                            product_desc = "|".join([
                                t.strip() for t in format_tag.find_all(text=True, recursive=False)
                            ]) if format_tag else None
                        except:
                            product_desc = None
            
                        try:
                            verified_purchase = ind_review.find('span', attrs={"data-hook": "avp-badge"}) is not None
                        except:
                            verified_purchase = False
            
                        try:
                            review_text = ind_review.find('span', attrs={"data-hook": "review-body"}) \
                                                     .find('span', class_=None).text.strip()
                        except:
                            review_text = None
            
                        try:
                            helpful = ind_review.find('span', attrs={"data-hook": "helpful-vote-statement"})
                            helpful_votes = helpful.text.split()[0].strip() if helpful else "0"
                        except:
                            helpful_votes = None
            
                        product_detail_list.append([asin, model_number, manufacturer_name, generic_name,
                            prod_title, price, overall_rating, rating_count, review_count, bought_last_month, product_link, customer_say, insight,
                            rating, review_title, review_date,
                            product_desc, verified_purchase, review_text, helpful_votes
                        ])
            
                    try:
                        next_li = browser.find_element(By.CLASS_NAME, 'a-last')
                        if "a-disabled" in next_li.get_attribute("class"):
                            print("✅ Reached last page of reviews.")
                            break
                        else:
                            try:
                                next_button = next_li.find_element(By.TAG_NAME, 'a')
                                browser.execute_script("arguments[0].click();", next_button)
                                time.sleep(random.uniform(2, 4))
                            except Exception as e:
                                print("⚠️ Couldn't click the next button:", e)
                                break
                    except NoSuchElementException:
                        print("✅ No pagination (only one page of reviews).")
                        break
            except:
                print("⚠️ Error in processing product:", e)
                continue
                
except KeyboardInterrupt:
    print("\n🛑 Scraping stopped by user (Ctrl+C).")
except Exception as e:
    print(f"\n❌ Unexpected Error: {e}")


> ⚠️ Output of the above cell has been removed to keep the notebook readable. Below is sample output.

Enter product you want to search (like Mobile, laptop ....) -  laptop

✅ You can search by Index or Brand Name

Index                 Brand Name  
    0                        all  
    1                         HP  
    2                     Lenovo  
    3                       ASUS  
    4                       acer  
    5                       Dell  
    6                      Apple  
    7                        MSI  
    8                    Samsung  
    9                  Microsoft  
   10                   GIGABYTE  
   11                      Dyazo  
   12                    Lapcare  
   13                  ZEBRONICS  
   14           ah arctic hunter  
   15              amazon basics  
   16                        rts  
   17                  Stuffcool  
   18                  uppercase  
   19                   Logitech  
   20                     COSMUS  
   21        HAMMONDS FLYCATCHER  
   22                   MOKOBARA  
   23                     Sounce  
   24                 Portronics  
   25                      Shure  
   26                    Crucial  
   27               GADGETS WRAP  
   28                       EUME  
   29                   WildHorn  
   30              THE CLOWNFISH  
   31 LS LAPSTER Quality Assured  
   32             SWISS MILITARY  
   33                    Ambrane  
   34                     Belkin  
   35                       BenQ  
   36                      Anker  
   37                     Tukzer  
   38                Ant Esports  
   39                       Gear  
   40                     Safari  

Enter Index or Brand Name you want to search -  apple, hp, lenovo, msi, dell  
Brand Name -  apple  
Brand Name -  Lenovo  
Brand Name -  Dell  
Brand Name -  HP  
Brand Name -  MSI  

last page 20  
Page Url https://www.amazon.in/s?k=laptop&rh=p_123%3A110955%257C241862%257C308445%257C378555%257C391242&dc&qid=1752467357&rnid=91049095031&ref=sr_nr_p_123_7&ds=v1%3Au0BxVt3pN1g%2BfNoWZv84AIVlLFAt3%2BlXjGpykMocOC4&page=1  

ASIN, Model Number, manufacturer_na for t in format_tag.find_all(text=True, recursive=False)  
✅ Reached last page of reviews.  

ASIN, Model Number, manufacturer_name, generic_name B0F8P4Y7VF C08Q6PA HP Laptop  
❌ No valid Review link found in this product.  

ASIN, Model Number, manufacturer_name, generic_name B0DP7BZ8FD 14IAH8 Lenovo Laptop  
✅ Reached last page of reviews.  

ASIN, Model Number, manufacturer_name, generic_name B0D3HG5CMG 9D3M8PA HP Laptop  
✅ Reached last page of reviews.  

ASIN, Model Number, manufacturer_name, generic_name B0FCD9LKLZ 250 G9 HP Business Laptop  
✅ No pagination (only one page of reviews).  

ASIN, Model Number, manufacturer_name, generic_name B0F32ZBDNJ Intel Core i7-1255U Upto 4.70Ghz Lenovo Laptop  
✅ Reached last page of reviews.  

ASIN, Model Number, manufacturer_name, generic_name B08N5W4NNB MGN63HN/A Apple Laptop  
✅ Reached last page of reviews.  

ASIN, Model Number, manufacturer_name, generic_name B0DZJ262NX 7535HS Lenovo Laptop  
✅ No pagination (only one page of reviews).  

ASIN, Model Number, manufacturer_name, generic_name B0FFGWWHBP 240 G10 hp Notebook Laptop  
✅ No pagination (only one page of reviews).  

ASIN, Model Number, manufacturer_name, generic_name B0F2162VGQ 15IRH10 Lenovo Laptop  
✅ Reached last page of reviews.


In [22]:
brand_df

,Index,Brand Name,Url
0,0,all,https://www.amazon.in/s?k=laptop
1,1,HP,https://www.amazon.in/s?k=laptop&rh=p_123%3A30...
2,2,Lenovo,https://www.amazon.in/s?k=laptop&rh=p_123%3A39...
3,3,ASUS,https://www.amazon.in/s?k=laptop&rh=p_123%3A21...
4,4,acer,https://www.amazon.in/s?k=laptop&rh=p_123%3A24...
5,5,Dell,https://www.amazon.in/s?k=laptop&rh=p_123%3A24...
6,6,Apple,https://www.amazon.in/s?k=laptop&rh=p_123%3A11...
7,7,MSI,https://www.amazon.in/s?k=laptop&rh=p_123%3A37...
8,8,Samsung,https://www.amazon.in/s?k=laptop&rh=p_123%3A46...
9,9,Microsoft,https://www.amazon.in/s?k=laptop&rh=p_123%3A42...


## 📊 Creating DataFrame from Scraped Data

After scraping product and review information

In [7]:

columns_1 = ["ASIN", "Model Number", "Manufacturer Name", "Generic Name",
    "Product Title", "Price", "Overall Rating", "Rating Count", "Review Count", "Bought L Month", "Product Link", "Customer Say", "Insight",
    "Individual Rating", "Review Title", "Review Date",
    "Product Description", "Verified Purchase", "Review Text", "Helpful Votes"
]

df = pd.DataFrame(data = product_detail_list, columns=columns_1)


In [8]:
df.head(5)

,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,Bought L Month,Product Link,Customer Say,Insight,Individual Rating,Review Title,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes
0,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","89,990.",4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,5.0,Fast and Powerful!,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,I recently purchased the MacBook M4 with 512GB...,38
1,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","89,990.",4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,5.0,Elevate Your Productivity with the Perfect Laptop,Reviewed in India on 4 April 2025,Colour: Midnight|Size: 16GB Unified Memory|Sty...,True,I placed the order on March 28th and received ...,118
2,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","89,990.",4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,5.0,"Great build quality, good for everyday/office use",Reviewed in India on 9 May 2025,Colour: Sky Blue|Size: 16GB Unified Memory|Sty...,True,New to the apple ecosystem. Every single apple...,4
3,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","89,990.",4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,5.0,Sleek and super fast,Reviewed in India on 4 July 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,"Very easy to use , has met my expectations",One
4,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","89,990.",4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,None,None,5.0,Overwhelmed with my purchase,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,"Just got it 2 days ago. So far, just overwhelm...",25


In [9]:
# df.to_csv('1. product_laptop.csv', index=False)